In [ ]:
import calendar
from datetime import date, timedelta
from django.db.models import Sum, Q
from django_pandas.io import read_frame
from IPython.display import Markdown as md

end_date = date.today().replace(day=1) - timedelta(days=1) # end of last month
begin_date = end_date.replace(day=1, month=end_date.month - 2) # 1st day of quarter
q = {m: 'Q1' for m in [1, 2, 3]}
q.update({m: 'Q2' for m in [4, 5, 6]})
q.update({m: 'Q3' for m in [7, 8, 9]})
q.update({m: 'Q4' for m in [10, 11, 12]})
md(f'# {q[begin_date.month]} Performance')

In [ ]:
def add_click_block_rate(df):
    df['click rate'] = 100. * df['clicks'] / df['impressions']
    df['block rate'] = 100. * df['blocks'] / df['impressions']

In [ ]:
format_dict = {k: '{:,.0f}' for k in ('blocks', 'clicks', 'impressions')}
format_dict.update({k: '{:.2f}%' for k in ('block rate', 'click rate')})

## Categories

In [ ]:
FXA_PAID_BEGIN_DATE = date(2019, 12, 9)
if end_date < FXA_PAID_BEGIN_DATE: 
    category_queryset = Category.objects.exclude(name='FxA-Paid')
else:
    category_queryset = Category.objects.all()

date_filter = Q(snippets__jobs__dailyjobmetrics__date__gte=begin_date,
                snippets__jobs__dailyjobmetrics__date__lte=end_date)

categories = read_frame(
    category_queryset.annotate(
        clicks=Sum('snippets__jobs__dailyjobmetrics__clicks', filter=date_filter),
        blocks=Sum('snippets__jobs__dailyjobmetrics__blocks', filter=date_filter),
        impressions=Sum('snippets__jobs__dailyjobmetrics__impressions', filter=date_filter)))

add_click_block_rate(categories)
columns = ['name', 'impressions', 'clicks', 'click rate', 'blocks', 'block rate']
categories[columns].style.format(format_dict).hide_index()

## Channels

In [ ]:
channels = read_frame(
    DailyChannelMetrics.objects.filter(
        date__gte=begin_date, date__lte=end_date)).groupby('channel').sum()
add_click_block_rate(channels)

columns.remove('name')
channels.sort_values('impressions', ascending=False)[columns].style.format(format_dict)

## Countries

In [ ]:
countries = read_frame(
    DailyCountryMetrics.objects.filter(
        date__gte=begin_date, date__lte=end_date)).groupby('country').sum()
add_click_block_rate(countries)
countries.nlargest(10, 'impressions')[columns].style.format(format_dict)